# 第七章 型態、方法與介面 - Part 4: 進階應用

本部分將探討 Go 語言介面系統的進階應用，包括依賴管理、Wire 模式，以及 Go 對物件導向程式設計的現代詮釋。

## 隱藏介面讓相依關係容易進行

介面應該定義在使用方，而不是實現方。這樣可以減少依賴，提高模組化程度。

### 依賴反轉原則
- 高層模組不應該依賴低層模組
- 兩者都應該依賴抽象
- 抽象不應該依賴細節

### 實作策略
- 在需要的地方定義最小介面
- 避免大而全的介面
- 介面隔離原則

### 優勢
- **降低耦合**: 模組間依賴減少
- **易於測試**: 方便建立 mock
- **靈活擴展**: 新實現不影響使用方
- **清晰邊界**: 明確的模組職責

In [ ]:
package main

import "fmt"

// ======================================================================
// 壞的設計：在實現方定義介面
// ======================================================================

// database 套件定義了介面（不好的做法）
/*
package database

type UserRepository interface {
    GetUser(id int) (*User, error)
    SaveUser(user *User) error
    DeleteUser(id int) error
    ListUsers() ([]*User, error)
    GetUserByEmail(email string) (*User, error)
    UpdateUser(user *User) error
}

type MySQLUserRepository struct {}
// 實現所有方法...
*/

// ======================================================================
// 好的設計：在使用方定義介面
// ======================================================================

// service 套件只需要它用到的方法
type UserGetter interface {
    GetUser(id int) (*User, error)
}

type UserSaver interface {
    SaveUser(user *User) error
}

// UserService 只依賴它需要的介面
type UserService struct {
    userGetter UserGetter
    userSaver  UserSaver
}

func NewUserService(getter UserGetter, saver UserSaver) *UserService {
    return &UserService{
        userGetter: getter,
        userSaver:  saver,
    }
}

func (us *UserService) ProcessUser(id int, name string) error {
    // 只使用需要的方法
    user, err := us.userGetter.GetUser(id)
    if err != nil {
        return err
    }
    
    user.Name = name
    return us.userSaver.SaveUser(user)
}

// ======================================================================
// 另一個服務有不同的需求
// ======================================================================

type UserLister interface {
    ListUsers() ([]*User, error)
}

type EmailValidator interface {
    GetUserByEmail(email string) (*User, error)
}

type AdminService struct {
    userLister     UserLister
    emailValidator EmailValidator
}

func NewAdminService(lister UserLister, validator EmailValidator) *AdminService {
    return &AdminService{
        userLister:     lister,
        emailValidator: validator,
    }
}

func (as *AdminService) ValidateEmail(email string) bool {
    _, err := as.emailValidator.GetUserByEmail(email)
    return err == nil
}

// ======================================================================
// 實現方：可以實現多個小介面
// ======================================================================

type User struct {
    ID    int
    Name  string
    Email string
}

// 實現所有小介面
type DatabaseRepository struct {
    users map[int]*User
}

func NewDatabaseRepository() *DatabaseRepository {
    return &DatabaseRepository{
        users: make(map[int]*User),
    }
}

// 實現 UserGetter
func (db *DatabaseRepository) GetUser(id int) (*User, error) {
    if user, exists := db.users[id]; exists {
        return user, nil
    }
    return nil, fmt.Errorf("user not found: %d", id)
}

// 實現 UserSaver
func (db *DatabaseRepository) SaveUser(user *User) error {
    db.users[user.ID] = user
    fmt.Printf("Saved user: %+v\n", user)
    return nil
}

// 實現 UserLister
func (db *DatabaseRepository) ListUsers() ([]*User, error) {
    users := make([]*User, 0, len(db.users))
    for _, user := range db.users {
        users = append(users, user)
    }
    return users, nil
}

// 實現 EmailValidator
func (db *DatabaseRepository) GetUserByEmail(email string) (*User, error) {
    for _, user := range db.users {
        if user.Email == email {
            return user, nil
        }
    }
    return nil, fmt.Errorf("user not found with email: %s", email)
}

// ======================================================================
// 測試友好的設計
// ======================================================================

type MockUserGetter struct {
    users map[int]*User
}

func (m *MockUserGetter) GetUser(id int) (*User, error) {
    if user, exists := m.users[id]; exists {
        return user, nil
    }
    return nil, fmt.Errorf("mock: user not found")
}

type MockUserSaver struct {
    savedUsers []*User
}

func (m *MockUserSaver) SaveUser(user *User) error {
    m.savedUsers = append(m.savedUsers, user)
    return nil
}

func main() {
    fmt.Println("=== 依賴反轉原則示例 ===")
    
    // 創建實現
    repo := NewDatabaseRepository()
    
    // 先添加一些測試資料
    repo.SaveUser(&User{ID: 1, Name: "Alice", Email: "alice@example.com"})
    repo.SaveUser(&User{ID: 2, Name: "Bob", Email: "bob@example.com"})
    
    // 創建服務，只注入需要的介面
    userService := NewUserService(repo, repo)  // 同一個實現，不同的介面
    adminService := NewAdminService(repo, repo)
    
    // 使用服務
    err := userService.ProcessUser(1, "Alice Updated")
    if err != nil {
        fmt.Printf("Error: %v\n", err)
    }
    
    isValid := adminService.ValidateEmail("bob@example.com")
    fmt.Printf("Email validation result: %t\n", isValid)  // 輸出: true
    
    fmt.Println("\n=== 測試友好設計 ===")
    
    // 使用 mock 進行測試
    mockGetter := &MockUserGetter{
        users: map[int]*User{
            1: {ID: 1, Name: "Test User", Email: "test@example.com"},
        },
    }
    mockSaver := &MockUserSaver{}
    
    testService := NewUserService(mockGetter, mockSaver)
    err = testService.ProcessUser(1, "Updated Test User")
    if err != nil {
        fmt.Printf("Test error: %v\n", err)
    } else {
        fmt.Printf("Test successful, saved users: %d\n", len(mockSaver.savedUsers))  // 輸出: 1
    }
    
    fmt.Println("\n=== 設計原則優勢 ===")
    fmt.Println("✓ 介面隔離：每個服務只依賴需要的方法")
    fmt.Println("✓ 依賴反轉：高層不依賴低層實現")  
    fmt.Println("✓ 單一職責：每個介面有明確職責")
    fmt.Println("✓ 易於測試：可以輕鬆 mock 依賴")
    fmt.Println("✓ 靈活擴展：新實現不影響現有程式碼")
}

## Wire

Wire 是 Google 開源的 Go 依賴注入工具，它使用程式碼生成來自動創建依賴關係。雖然 Wire 不是 Go 標準程式庫的一部分，但它體現了 Go 介面和依賴反轉的最佳實踐。

### 依賴注入的必要性
在複雜的應用程式中，物件之間存在複雜的依賴關係。傳統的創建方式可能導致：

**問題**:
- 硬編碼的依賴關係
- 難以進行單元測試
- 程式碼重複
- 缺乏靈活性

**解決方案**:
Wire 提供了一種類型安全的方式來處理依賴注入，在編譯時生成依賴關係，避免了執行時反射的開銷。

### Wire 的基本概念
**Provider（提供者）**: 函式，負責創建和配置物件
**Wire Set**: 一組相關的 Provider
**Injector（注入器）**: 將 Provider 連接起來創建最終物件的函式

In [ ]:
package main

import "fmt"

// ======================================================================
// Wire 風格的依賴注入模式（手動實現）
// ======================================================================

// 定義介面
type Logger interface {
    Log(message string)
}

type Database interface {
    Store(key, value string) error
    Get(key string) (string, error)
}

type Config interface {
    GetDatabaseURL() string
    GetLogLevel() string
}

// ======================================================================
// 具體實現
// ======================================================================

type ConsoleLogger struct {
    level string
}

func (c *ConsoleLogger) Log(message string) {
    fmt.Printf("[%s] %s\n", c.level, message)
}

type MemoryDatabase struct {
    data map[string]string
}

func (m *MemoryDatabase) Store(key, value string) error {
    m.data[key] = value
    return nil
}

func (m *MemoryDatabase) Get(key string) (string, error) {
    value, exists := m.data[key]
    if !exists {
        return "", fmt.Errorf("key not found: %s", key)
    }
    return value, nil
}

type AppConfig struct {
    databaseURL string
    logLevel    string
}

func (a *AppConfig) GetDatabaseURL() string {
    return a.databaseURL
}

func (a *AppConfig) GetLogLevel() string {
    return a.logLevel
}

// ======================================================================
// 業務服務
// ======================================================================

type UserService struct {
    logger   Logger
    database Database
    config   Config
}

func (u *UserService) CreateUser(name, email string) error {
    u.logger.Log(fmt.Sprintf("Creating user: %s", name))
    
    err := u.database.Store(name, email)
    if err != nil {
        u.logger.Log(fmt.Sprintf("Failed to store user: %v", err))
        return err
    }
    
    u.logger.Log(fmt.Sprintf("User created successfully: %s", name))
    return nil
}

func (u *UserService) GetUserEmail(name string) (string, error) {
    u.logger.Log(fmt.Sprintf("Getting user email: %s", name))
    return u.database.Get(name)
}

// ======================================================================
// Wire 風格的 Provider 函式
// ======================================================================

// Provider: 創建 Config
func NewConfig() Config {
    return &AppConfig{
        databaseURL: "memory://localhost",
        logLevel:    "INFO",
    }
}

// Provider: 創建 Logger
func NewLogger(config Config) Logger {
    return &ConsoleLogger{
        level: config.GetLogLevel(),
    }
}

// Provider: 創建 Database
func NewDatabase(config Config) Database {
    // 在實際應用中，會根據 config.GetDatabaseURL() 創建不同的資料庫實現
    return &MemoryDatabase{
        data: make(map[string]string),
    }
}

// Provider: 創建 UserService
func NewUserService(logger Logger, database Database, config Config) *UserService {
    return &UserService{
        logger:   logger,
        database: database,
        config:   config,
    }
}

// ======================================================================
// Wire 風格的 Injector（手動實現）
// ======================================================================

// 這個函式相當於 Wire 生成的 injector
func InitializeUserService() (*UserService, error) {
    // 按依賴順序創建物件
    config := NewConfig()
    logger := NewLogger(config)
    database := NewDatabase(config)
    userService := NewUserService(logger, database, config)
    
    return userService, nil
}

// ======================================================================
// Wire Set 概念（手動組織）
// ======================================================================

// 模擬 Wire Set：相關的 Provider 組合
type ProviderSet struct {
    ConfigProvider   func() Config
    LoggerProvider   func(Config) Logger
    DatabaseProvider func(Config) Database
    ServiceProvider  func(Logger, Database, Config) *UserService
}

var DefaultProviderSet = ProviderSet{
    ConfigProvider:   NewConfig,
    LoggerProvider:   NewLogger,
    DatabaseProvider: NewDatabase,
    ServiceProvider:  NewUserService,
}

// 使用 ProviderSet 的 injector
func InitializeWithProviderSet(providerSet ProviderSet) (*UserService, error) {
    config := providerSet.ConfigProvider()
    logger := providerSet.LoggerProvider(config)
    database := providerSet.DatabaseProvider(config)
    userService := providerSet.ServiceProvider(logger, database, config)
    
    return userService, nil
}

// ======================================================================
// 測試用的 Provider Set
// ======================================================================

type MockLogger struct {
    messages []string
}

func (m *MockLogger) Log(message string) {
    m.messages = append(m.messages, message)
}

func NewMockLogger(config Config) Logger {
    return &MockLogger{messages: make([]string, 0)}
}

var TestProviderSet = ProviderSet{
    ConfigProvider:   NewConfig,
    LoggerProvider:   NewMockLogger,  // 使用 Mock Logger
    DatabaseProvider: NewDatabase,
    ServiceProvider:  NewUserService,
}

func main() {
    fmt.Println("=== Wire 風格依賴注入示例 ===")
    
    // 使用手動 injector
    userService, err := InitializeUserService()
    if err != nil {
        fmt.Printf("Failed to initialize: %v\n", err)
        return
    }
    
    // 使用服務
    err = userService.CreateUser("Alice", "alice@example.com")
    if err != nil {
        fmt.Printf("Error creating user: %v\n", err)
        return
    }
    
    email, err := userService.GetUserEmail("Alice")
    if err != nil {
        fmt.Printf("Error getting user email: %v\n", err)
        return
    }
    
    fmt.Printf("Retrieved email: %s\n", email)  // 輸出: Retrieved email: alice@example.com
    
    fmt.Println("\n=== 使用 Provider Set ===")
    
    // 使用默認 Provider Set
    service1, _ := InitializeWithProviderSet(DefaultProviderSet)
    service1.CreateUser("Bob", "bob@example.com")
    
    fmt.Println("\n=== 測試友好的設計 ===")
    
    // 使用測試 Provider Set
    testService, _ := InitializeWithProviderSet(TestProviderSet)
    testService.CreateUser("TestUser", "test@example.com")
    
    // 檢查 Mock Logger
    if mockLogger, ok := testService.logger.(*MockLogger); ok {
        fmt.Printf("Mock logger captured %d messages\n", len(mockLogger.messages))  // 輸出消息數量
        for _, msg := range mockLogger.messages {
            fmt.Printf("  - %s\n", msg)
        }
    }
    
    fmt.Println("\n=== Wire 的優勢 ===")
    fmt.Println("1. 編譯時安全：在編譯時檢查依賴關係")
    fmt.Println("2. 零執行時開銷：沒有反射或容器開銷")  
    fmt.Println("3. 明確的依賴：清楚地看到依賴關係")
    fmt.Println("4. 易於測試：容易替換實現進行測試")
    fmt.Println("5. 型態安全：編譯器檢查型態匹配")
    fmt.Println("6. 程式碼生成：自動化繁瑣的依賴注入程式碼")
    
    fmt.Println("\n=== 與傳統 DI 容器的區別 ===")
    fmt.Println("傳統 DI 容器：")
    fmt.Println("- 執行時反射")
    fmt.Println("- 配置檔案驅動")
    fmt.Println("- 執行時錯誤")
    fmt.Println("- 效能開銷")
    
    fmt.Println("\nWire 方式：")
    fmt.Println("- 編譯時程式碼生成")
    fmt.Println("- 程式碼驅動")
    fmt.Println("- 編譯時錯誤")
    fmt.Println("- 零執行時開銷")
}

## Go 不是非物件導向的語言（這是好事）

Go 經常被誤解為「不支持物件導向程式設計」的語言，但這個說法並不完全正確。Go 支持物件導向的核心概念，只是採用了不同的方式來實現，這種方式往往更簡潔、更靈活。

### Go 對物件導向的獨特詮釋

Go 支持物件導向程式設計的三個核心概念，但採用了現代化的方式：

**封裝 (Encapsulation)**:
- 通過首字母大小寫控制可見性
- 結構體和方法提供資料和行為的封裝
- 介面定義抽象邊界

**多型 (Polymorphism)**:
- 通過介面實現多型
- 隱式介面實現，更加靈活
- 類型斷言和 type switch 提供運行時多型

**組合 (Composition over Inheritance)**:
- 使用組合而非繼承
- 結構體嵌入提供代碼重用
- 更靈活的代碼組織方式

### 與傳統 OOP 的差異

**沒有類的概念**:
- 使用結構體 (struct) 代替類
- 方法附加到型態上，而不是定義在類中
- 更直接的型態系統

**沒有繼承**:
- 使用組合代替繼承
- 避免了深層繼承鏈的複雜性
- 更清晰的依賴關係

**隱式介面**:
- 不需要明確聲明實現關係
- Duck Typing 的類型安全版本
- 更靈活的介面設計

In [ ]:
package main

import "fmt"

// ======================================================================
// 封裝 (Encapsulation) - 通過可見性控制
// ======================================================================

// BankAccount 展示封裝：私有欄位，公開方法
type BankAccount struct {
    owner   string  // 私有欄位（小寫開頭）
    balance float64 // 私有欄位
}

// 公開的建構函式
func NewBankAccount(owner string, initialBalance float64) *BankAccount {
    return &BankAccount{
        owner:   owner,
        balance: initialBalance,
    }
}

// 公開方法，控制對私有資料的存取
func (ba *BankAccount) GetOwner() string {
    return ba.owner
}

func (ba *BankAccount) GetBalance() float64 {
    return ba.balance
}

func (ba *BankAccount) Deposit(amount float64) error {
    if amount <= 0 {
        return fmt.Errorf("deposit amount must be positive")
    }
    ba.balance += amount
    return nil
}

func (ba *BankAccount) Withdraw(amount float64) error {
    if amount <= 0 {
        return fmt.Errorf("withdraw amount must be positive")
    }
    if ba.balance < amount {
        return fmt.Errorf("insufficient funds")
    }
    ba.balance -= amount
    return nil
}

// ======================================================================
// 多型 (Polymorphism) - 通過介面實現
// ======================================================================

// 定義 Shape 介面
type Shape interface {
    Area() float64
    Perimeter() float64
    GetInfo() string
}

// 不同的形狀實現相同的介面
type Circle struct {
    Radius float64
}

func (c Circle) Area() float64 {
    return 3.14159 * c.Radius * c.Radius
}

func (c Circle) Perimeter() float64 {
    return 2 * 3.14159 * c.Radius
}

func (c Circle) GetInfo() string {
    return fmt.Sprintf("Circle with radius %.2f", c.Radius)
}

type Rectangle struct {
    Width, Height float64
}

func (r Rectangle) Area() float64 {
    return r.Width * r.Height
}

func (r Rectangle) Perimeter() float64 {
    return 2 * (r.Width + r.Height)
}

func (r Rectangle) GetInfo() string {
    return fmt.Sprintf("Rectangle %.2f x %.2f", r.Width, r.Height)
}

type Triangle struct {
    Base, Height float64
}

func (t Triangle) Area() float64 {
    return 0.5 * t.Base * t.Height
}

func (t Triangle) Perimeter() float64 {
    // 簡化計算，假設是直角三角形
    hypotenuse := t.Base*t.Base + t.Height*t.Height
    return t.Base + t.Height + hypotenuse
}

func (t Triangle) GetInfo() string {
    return fmt.Sprintf("Triangle with base %.2f and height %.2f", t.Base, t.Height)
}

// 多型函式：接受任何 Shape
func PrintShapeDetails(shapes []Shape) {
    fmt.Println("=== Shape Details ===")
    totalArea := 0.0
    
    for i, shape := range shapes {
        fmt.Printf("%d. %s\n", i+1, shape.GetInfo())
        fmt.Printf("   Area: %.2f, Perimeter: %.2f\n", shape.Area(), shape.Perimeter())
        totalArea += shape.Area()
    }
    
    fmt.Printf("Total area: %.2f\n", totalArea)
}

// ======================================================================
// 組合 (Composition) - 結構體嵌入
// ======================================================================

// 基礎能力
type Drawable interface {
    Draw() string
}

type Movable interface {
    Move(x, y float64)
    GetPosition() (float64, float64)
}

// Position 提供移動能力
type Position struct {
    X, Y float64
}

func (p *Position) Move(x, y float64) {
    p.X = x
    p.Y = y
}

func (p Position) GetPosition() (float64, float64) {
    return p.X, p.Y
}

// Color 提供顏色資訊
type Color struct {
    R, G, B uint8
}

func (c Color) GetColorString() string {
    return fmt.Sprintf("RGB(%d,%d,%d)", c.R, c.G, c.B)
}

// DrawableCircle 組合多種能力
type DrawableCircle struct {
    Circle   // 嵌入 Circle，獲得 Shape 能力
    Position // 嵌入 Position，獲得 Movable 能力
    Color    // 嵌入 Color，獲得顏色能力
}

// 實現 Drawable 介面
func (dc DrawableCircle) Draw() string {
    x, y := dc.GetPosition()
    return fmt.Sprintf("Drawing %s at (%.1f, %.1f) with color %s", 
        dc.GetInfo(), x, y, dc.GetColorString())
}

// DrawableRectangle 同樣組合多種能力
type DrawableRectangle struct {
    Rectangle
    Position
    Color
}

func (dr DrawableRectangle) Draw() string {
    x, y := dr.GetPosition()
    return fmt.Sprintf("Drawing %s at (%.1f, %.1f) with color %s", 
        dr.GetInfo(), x, y, dr.GetColorString())
}

// ======================================================================
// 現代 OOP 的體現
// ======================================================================

// Canvas 展示如何使用 Go 的 OOP 特性
type Canvas struct {
    width, height float64
    objects       []Drawable
}

func NewCanvas(width, height float64) *Canvas {
    return &Canvas{
        width:   width,
        height:  height,
        objects: make([]Drawable, 0),
    }
}

func (c *Canvas) AddObject(obj Drawable) {
    c.objects = append(c.objects, obj)
}

func (c *Canvas) Render() {
    fmt.Printf("Rendering canvas (%.1f x %.1f):\n", c.width, c.height)
    for i, obj := range c.objects {
        fmt.Printf("%d. %s\n", i+1, obj.Draw())
    }
}

func main() {
    fmt.Println("=== Go 的現代 OOP 展示 ===")
    
    fmt.Println("\n1. 封裝 (Encapsulation)")
    account := NewBankAccount("Alice", 1000)
    fmt.Printf("Account owner: %s\n", account.GetOwner())
    fmt.Printf("Initial balance: %.2f\n", account.GetBalance())
    
    account.Deposit(500)
    fmt.Printf("After deposit: %.2f\n", account.GetBalance())
    
    err := account.Withdraw(200)
    if err != nil {
        fmt.Printf("Withdraw error: %v\n", err)
    } else {
        fmt.Printf("After withdrawal: %.2f\n", account.GetBalance())
    }
    
    fmt.Println("\n2. 多型 (Polymorphism)")
    shapes := []Shape{
        Circle{Radius: 5},
        Rectangle{Width: 4, Height: 6},
        Triangle{Base: 3, Height: 4},
    }
    PrintShapeDetails(shapes)
    
    fmt.Println("\n3. 組合 (Composition)")
    drawableCircle := DrawableCircle{
        Circle:   Circle{Radius: 3},
        Position: Position{X: 10, Y: 20},
        Color:    Color{R: 255, G: 0, B: 0},
    }
    
    drawableRectangle := DrawableRectangle{
        Rectangle: Rectangle{Width: 5, Height: 3},
        Position:  Position{X: 30, Y: 40},
        Color:     Color{R: 0, G: 255, B: 0},
    }
    
    // 移動物件
    drawableCircle.Move(15, 25)
    drawableRectangle.Move(35, 45)
    
    // 創建 Canvas 並渲染
    canvas := NewCanvas(100, 100)
    canvas.AddObject(drawableCircle)
    canvas.AddObject(drawableRectangle)
    canvas.Render()
    
    fmt.Println("\n=== Go OOP 的優勢 ===")
    fmt.Println("✓ 簡潔性：沒有複雜的繼承層次")
    fmt.Println("✓ 靈活性：組合比繼承更靈活")
    fmt.Println("✓ 性能：沒有虛函數表的開銷")
    fmt.Println("✓ 類型安全：編譯時檢查")
    fmt.Println("✓ 易測試：容易建立 mock 實現")
    fmt.Println("✓ 清晰的依賴關係")
    fmt.Println("✓ 避免了多重繼承的複雜性")
    
    fmt.Println("\n=== 現代 OOP 的演進趨勢 ===")
    fmt.Println("傳統 OOP → 現代 OOP:")
    fmt.Println("• 繼承 → 組合")
    fmt.Println("• 大型類 → 小型介面")  
    fmt.Println("• 緊耦合 → 鬆耦合")
    fmt.Println("• 複雜層次 → 扁平結構")
    fmt.Println("• 運行時多型 → 編譯時安全")
    
    fmt.Println("\nGo 完美體現了現代 OOP 的最佳實踐！")
}